In [8]:
########################################################################################
# library imports
########################################################################################

# disable warnings
import warnings
warnings.filterwarnings("ignore")

# generic libraries
from platform import python_version
import functools
import inspect
from collections import deque
import diskcache as dc
import hashlib
import numpy as np
import os
import datetime
import random
import matplotlib.pyplot as plt

# tensor flow / keras related libraries
import tensorflow as tf
import tensorflow_io as tfio
import tensorflow_hub as hub
from keras.utils import dataset_utils

# image processing related libraries
import librosa 

# audio processing libraries
import audiomentations
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

# print system information
print('Python Version           : ', python_version())
print('TensorFlow Version       : ', tf.__version__)
print('TensorFlow IO Version    : ', tfio.__version__)
print('Librosa Version          : ', librosa.__version__)
print('Audiomentations Version  : ', audiomentations.__version__)

Python Version           :  3.8.16
TensorFlow Version       :  2.10.1
TensorFlow IO Version    :  0.27.0
Librosa Version          :  0.9.2
Audiomentations Version  :  0.29.0


In [21]:
########################################################################################
# Create a DiskCache instance
# This cache will allow us store intermediate function results to speed up the 
# data processing pipeline
########################################################################################
cache = dc.Cache('d:\\test_cache\\', cull_limit=0, size_limit=10**9) 


########################################################################################
########################################################################################
def get_function_code(func):
    source_code = inspect.getsource(func)
    called_functions = []

    for name, obj in inspect.getmembers(func.__globals__):
        if inspect.isfunction(obj):
            called_functions.append(obj)

    return source_code, called_functions


########################################################################################
########################################################################################
def create_function_key(func, *args, **kwargs):
    partial_func = functools.partial(func, *args, **kwargs)
    func_name = partial_func.func.__name__
    func_module = partial_func.func.__module__
    args_repr = repr(partial_func.args)
    kwargs_repr = repr(sorted(partial_func.keywords.items()))

    source_code, called_functions = get_function_code(func)

    function_queue = deque(called_functions)
    while function_queue:
        current_function = function_queue.popleft()
        current_source_code, current_called_functions = get_function_code(current_function)
        source_code += current_source_code
        function_queue.extend(current_called_functions)

    key = f"{func_module}.{func_name}:{args_repr}:{kwargs_repr}:{source_code}"
    # Use hashlib to create a hash of the key for shorter and consistent length
    key_hash = hashlib.sha256(key.encode()).hexdigest()

    return key, key_hash, partial_func


########################################################################################
# Execute a function and cache the result
# If already executed, retrieve function output from the cache instead
########################################################################################
def execute_cached_function(func, *args, **kwargs):
    key_string,key,partial_func = create_function_key(func, *args, **kwargs)
    # Check if the result is in the cache
    if key in cache:
        result = cache[key]
        # print(f"Result loaded from cache key: {key}")
    else:
        # If not in cache, call the slow operation and store the result in cache
        result = partial_func()
        cache[key] = result
        #print(f"New result calculated and stored in cache key: {key}")
    return result

get_function_code(test_3)

('def test_3(a,b):\n    return 3*test_2(a,b)+a+b - test_1(78,67)\n', [])

In [18]:
def test_1(a,b):
    return 2*a + b

def test_2(a,b):
    return 2*a + b*test_1(a,b)

def test_3(a,b):
    return 3*test_2(a,b)+a+b - test_1(78,67)

create_function_key(test_3,6,7)

('__main__.test_3:(6, 7):[]:def test_3(a,b):\n    return 3*test_2(a,b)+a+b - test_1(78,67)\n',
 'cdec60ce1bf9809b36e9302c6ea525d443ad639c7b22205120061636147368b5',
 functools.partial(<function test_3 at 0x000001791DDECEE0>, 6, 7))

In [20]:
get_function_code(test_3)

('def test_3(a,b):\n    return 3*test_2(a,b)+a+b - test_1(78,67)\n', [])

In [6]:
import inspect
import ast

class FunctionCallCollector(ast.NodeVisitor):
    def __init__(self):
        self.called_function_names = set()

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            self.called_function_names.add(node.func.id)
        self.generic_visit(node)

def get_called_functions_from_globals(func, called_function_names):
    called_functions = []

    for name, obj in inspect.getmembers(func.__globals__):
        if inspect.isfunction(obj) and name in called_function_names:
            called_functions.append(obj)

    return called_functions

def get_all_source_code_recursively(func, visited=None):
    if visited is None:
        visited = set()

    if func in visited:
        return ""

    visited.add(func)

    source_code = inspect.getsource(func)
    tree = ast.parse(source_code)

    collector = FunctionCallCollector()
    collector.visit(tree)
    called_function_names = collector.called_function_names

    called_functions = get_called_functions_from_globals(func, called_function_names)

    for called_func in called_functions:
        source_code += get_all_source_code_recursively(called_func, visited)

    return source_code

# Usage example:

def foo(a, b):
    return a + b

def bar(a, b):
    return foo(a, b) * 2

def some_func(a, b):
    return bar(a, b) - 1

all_source_code = get_all_source_code_recursively(some_func)
print(all_source_code)


def some_func(a, b):
    return bar(a, b) - 1



In [7]:
import inspect
import ast

class FunctionCallCollector(ast.NodeVisitor):
    def __init__(self, available_functions):
        self.available_functions = available_functions
        self.called_function_names = set()

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            function_name = node.func.id
            if function_name in self.available_functions:
                self.called_function_names.add(function_name)
        self.generic_visit(node)

def get_called_functions(available_functions, called_function_names):
    called_functions = []

    for name in called_function_names:
        if name in available_functions:
            called_functions.append(available_functions[name])

    return called_functions

def get_all_source_code_recursively(func, available_functions=None, visited=None):
    if visited is None:
        visited = set()

    if available_functions is None:
        available_functions = {}

    if func in visited:
        return ""

    visited.add(func)

    source_code = inspect.getsource(func)
    tree = ast.parse(source_code)

    collector = FunctionCallCollector(available_functions)
    collector.visit(tree)
    called_function_names = collector.called_function_names

    called_functions = get_called_functions(available_functions, called_function_names)

    for called_func in called_functions:
        source_code += get_all_source_code_recursively(called_func, available_functions, visited)

    return source_code

# Usage example:

def foo(a, b):
    return a + b

def bar(a, b):
    return foo(a, b) * 2

def some_func(a, b):
    return bar(a, b) - 1

available_functions = {
    'foo': foo,
    'bar': bar,
    'some_func': some_func,
}

all_source_code = get_all_source_code_recursively(some_func, available_functions)
print(all_source_code)


def some_func(a, b):
    return bar(a, b) - 1
def bar(a, b):
    return foo(a, b) * 2
def foo(a, b):
    return a + b



In [3]:
import hashlib
import inspect
import ast
from typing import List

class FunctionCallCollector(ast.NodeVisitor):
    def __init__(self):
        self.called_functions = set()

def visit_Call(self, node):
    if isinstance(node.func, ast.Name):
        self.called_functions.add(node.func.id)
    self.generic_visit(node)

def get_function_code(func):
    source_code = inspect.getsource(func)
    return source_code

def collect_called_functions(source_code):
        tree = ast.parse(source_code)
        collector = FunctionCallCollector()
        collector.visit(tree)
        return collector.called_functions
    
def get_dependencies(func):
    code = get_function_code(func)
    dependencies = []

    dependencies = collect_called_functions(code)

    return list(dependencies)

def hash_code(code_list: List[str]):
    print(code_list)
    hasher = hashlib.sha256()
    for code in code_list:
        hasher.update(code.encode())
    return hasher.hexdigest()
      
def hash_function_signature(func):
    
    main_function_code = get_function_code(func)
    dependencies = get_dependencies(func)
    print(f'deps {dependencies}')
    code_list = [main_function_code]

    for dependency in dependencies:
        code_list.append(get_function_code(eval(dependency)))

    return hash_code(code_list)

def foo_1(a, b):
    return a + b

def foo_2(a, b):
    return a + b

def bar(a, b):
    return foo_1(a, b) * 2*foo_2(4,32)

def some_func(a, b):
    return bar(a, b) - 1

hash_value = hash_function_signature(some_func)
print("Hash value:", hash_value)


deps []
['def some_func(a, b):\n    return bar(a, b) - 1\n']
Hash value: 56ebb4ab38befe191714e46a9b6ae5dca6c8016537b388910158e214aa12c7c9
